In [1]:
from my_libs import *

robinhood = robingateway()


In [2]:
class option_trading():
    
    def __init__(self,robinhood ):
        self.trader = robinhood
    
    def __fetch_option_exp_date(self, symbol):
        """
        fetch option chain for instrument
        """
        url = "https://api.robinhood.com/options/chains/"
        params = {
            "equity_instrument_ids": self.trader.get_instrument(symbol)["id"].values[0],
            "state": "active",
            "tradability": "tradable"
        }
        data = self.trader.my_trader.session.get(url, params=params)
        return data.json()["results"][0]['expiration_dates']


    def __fetch_option_chain(self,symbol, p_c):
        p_c = p_c.lower()
        return pd.DataFrame(self.trader.my_trader.get_options(symbol,self.__fetch_option_exp_date(symbol), p_c))


    def __fetch_option_info(self,option_id):
        return self.trader.my_trader.get_option_market_data(option_id)
    
    def gen_leg(self,buy_sell,option_chain,position_effect ="open", ratio_quantity=1, direction = None):
        assert buy_sell in ["buy", "sell"]
        assert position_effect in ["open", "close"]
        return { "side": buy_sell,
        "option": option_chain["url"].values[0],
        "position_effect": position_effect,
        "ratio_quantity": ratio_quantity }

    def place_order(self, legs, direction, price, quantity):
        assert direction in ["debit", "credit"]

        return robinhood.my_trader.place_option_order(legs,quantity,price,direction)
    
    def get_exp_date(self,symbol,p_c):
        print ("Ascending")
        p_c = p_c.lower()
        return pd.Series(list(set(self.__fetch_option_chain(symbol,p_c).expiration_date))).sort_values().to_list()
    
    def get_strike(self, symbol,p_c,exp=None):
        if exp is None:
            exp = op.get_exp_date(symbol,pc)
            print (exp)
            exp= exp[input("Which exp date")]
        elif type(exp) == int:
            exp = op.get_exp_date(symbol,pc)[exp]
            print (exp)
        
        print ("Ascending")
        return list(pd.DataFrame(self.trader.my_trader.get_options(symbol,exp, p_c)).strike_price.astype(float).sort_values())
    
    
    def get_option(self, symbol,p_c,strike=None,exp=None):
        if exp is None:
            exp = op.get_exp_date(symbol,pc)
            print (exp)
            exp= exp[input("Which exp date")]
        elif type(exp) == int:
            exp = op.get_exp_date(symbol,pc)[exp]
            print (exp)
        
        result =  pd.DataFrame(self.trader.my_trader.get_options(symbol,exp, p_c))
        result.strike_price = result.strike_price.astype(float)
        if strike == None:
            return result
        
        return result[result.strike_price == strike]
    
    def match_price(data,target):
        pointer = len(data)/2
        def recur(pointer):   
            if data[pointer] - target ==0:
                return pointer
            elif data[pointer] - target >0:
                return pointer/2
            elif data[pointer] - target <0:
                return (len(data)-pointer)/2
            
        while pointer != 0 or data[pointer] - target != 0:
            pointer = recur(pointer)
            
        return data[pointer]
        
    def closest_strick_price(self,symbol,pc,target,exp = None):
        if exp is None:
            exp = op.get_exp_date(symbol,pc)
            print (exp)
            exp= exp[input("Which exp date")]
        elif type(exp) == int:
            exp = op.get_exp_date(symbol,pc)[exp]
            print (exp)
        
        data = self.get_strike(symbol,pc,exp)
        for i in range(len(data)):
            if data[i] - target < 0:
                continue
            elif data[i] - target == 0:
                return data[i]
            elif (target - data[i-1]) < (target - data[i]):
                
                return data[i-1]
            else:
                return data[i]
    def get_atp(self,symbol,pc,exp=None):
        if exp is None:
            exp = op.get_exp_date(symbol,pc)
            print (exp)
            exp= exp[input("Which exp date")]
        elif type(exp) == int:
            exp = op.get_exp_date(symbol,pc)[exp]
            print (exp)
        
        return float(self.closest_strick_price(symbol,pc,da.get_quote_yahoo(symbol).price.values[0],exp))
        
        
    def get_p_oc(self,symbol,pc,strike,exp = None):
        if exp is None:
            exp = op.get_exp_date(symbol,pc)
            print (exp)
            exp= exp[input("Which exp date")]
        elif type(exp) == int:
            exp = op.get_exp_date(symbol,pc)[exp]
            print (exp)
        
        cid = op.get_option(symbol,pc,exp=exp)
        cid.strike_price = cid.strike_price.apply(lambda x: float(x))
        cid = cid[cid.strike_price == strike ]
        
        return cid
    
    def get_atp_oc(self,symbol,pc,exp=None):
        if exp is None:
            exp = op.get_exp_date(symbol,pc)
            print (exp)
            exp= exp[input("Which exp date")]
        elif type(exp) == int:
            exp = op.get_exp_date(symbol,pc)[exp]
            print (exp)
        
        
        cid = op.get_option(symbol,pc,exp=exp)
        cid.strike_price = cid.strike_price.apply(lambda x: float(x))
        cid = cid[cid.strike_price == float(self.get_atp(symbol,pc,exp))]
        
        return cid
    
    def get_option_price(self,symbol,pc,exp): 
        return self.__fetch_option_info(self.get_atp_oc(symbol,pc,exp).iloc[0]["id"])
    
    def all_the_way_leg_atp(self,symbol,pc,buy_sell,effect,exp= None):
        if exp is None:
            exp = op.get_exp_date(symbol,pc)
            print (exp)
            exp= exp[input("Which exp date")]
        elif type(exp) == int:
            exp = op.get_exp_date(symbol,pc)[exp]
            print (exp)
            
        oc = self.get_atp_oc(symbol,pc,exp)

        self.get_atp(symbol,pc,exp=exp)

        

        return self.gen_leg(buy_sell,oc,position_effect = effect)
    
    
    def all_the_way_leg(self,symbol,pc,buy_sell,effect,strike,exp= None):
        if exp is None:
            exp = op.get_exp_date(symbol,pc)
            print (exp)
            exp= exp[input("Which exp date")]
        elif type(exp) == int:
            exp = op.get_exp_date(symbol,pc)[exp]
            print (exp)
            
        oc = self.get_p_oc(symbol,pc,strike,exp)

        self.get_atp(symbol,pc,exp=exp)


        return self.gen_leg(buy_sell,oc,position_effect = effect)
    
    def get_bid_price(self,symbol,pc,exp=None, strike = 'atp'):
        if exp is None:
            exp = op.get_exp_date(symbol,pc)
            print (exp)
            exp= exp[input("Which exp date")]
        elif type(exp) == int:
            exp = op.get_exp_date(symbol,pc)[exp]
            print (exp)
        if strike == 'atp':
            o = self.get_option(symbol,pc,strike =self.get_atp(symbol,pc,exp),exp=exp)
        else:
            o = self.get_option(symbol,pc,strike,exp)
#         return o
        return float(self.__fetch_option_info(o.iloc[0]["id"])["bid_price"])

In [3]:
op = option_trading(robinhood)

In [10]:
dd =op.get_bid_price("VIXY",'put')

Ascending
[u'2019-12-13', u'2019-12-27', u'2020-01-03', u'2020-01-10', u'2020-01-24']
Which exp date2
Ascending


In [11]:
dd

1.2

In [11]:
op.fetch_option_info(dd)

AttributeError: option_trading instance has no attribute 'fetch_option_info'

In [15]:

op.all_the_way_leg_atp("SPY",'call','buy','open',2)

# op.all_the_way_leg("VIXY",'call','buy','open',2)


Ascending
2019-12-23
Ascending
Ascending


{'option': u'https://api.robinhood.com/options/instruments/860c7ee0-142c-49d9-a545-1c753ef42eae/',
 'position_effect': 'open',
 'ratio_quantity': 1,
 'side': 'buy'}

In [17]:
robinhood.my_trader.place_option_order([op.all_the_way_leg_atp("VIXY",'call','sell','open',2)],1,op.get_bid_price("VIXY",'call',exp =2),'credit')


# submit(robinhood, direction, legs, price, quantity, time_in_force, trigger, order_type, run_validations=False)

Ascending
2020-01-03
Ascending
Ascending
Ascending
2020-01-03
Ascending


{u'detail': u'This order introduces infinite risk.'}

In [16]:
[op.get_bid_price("SPY",'call',2),op.get_bid_price("VIXY",'call',2)]

5.15

In [133]:
op.get_option("SPY","call",op.closest_strick_price("SPY","call",305))

Ascending
[u'2019-12-16', u'2019-12-20', u'2019-12-23', u'2020-01-13', u'2020-01-24']
Which exp date0
Ascending
Ascending
[u'2019-12-16', u'2019-12-20', u'2019-12-23', u'2020-01-13', u'2020-01-24']
Which exp date0


,chain_id,chain_symbol,created_at,expiration_date,id,issue_date,min_ticks,rhs_tradability,state,strike_price,tradability,type,updated_at,url
64,c277b118-58d9-4060-8dc5-a3b5898955cb,SPY,2019-11-08T01:05:43.983924Z,2019-12-16,44e5143b-1f9f-4090-acba-25c6ad81b850,2005-01-06,"{u'cutoff_price': u'0.00', u'below_tick': u'0....",untradable,active,305.0,tradable,call,2019-11-08T01:05:43.983931Z,https://api.robinhood.com/options/instruments/...
